# ChatFeed 聊天消息流组件

中层级的布局组件，用于管理一系列 ChatMessage 消息组件。该组件提供后端方法来：
- 发送(附加)消息到聊天记录
- 将字符流式显示到最近的 ChatMessage 中
- 当用户发送消息时执行回调
- 撤销多条 ChatMessage 消息
- 清空所有 ChatMessage 消息

底层实现为`panel.chat.ChatFeed`，参数基本一致，参考文档：https://panel.holoviz.org/reference/chat/ChatFeed.html


In [ ]:
##ignore
%load_ext vuepy
from panel_vuepy import vpanel


## 基本用法

基本的消息流组件可以不带任何参数初始化：


In [ ]:
%%vuepy_run --plugins vpanel --show-code
<template>
<PnChatFeed ref="chat" />
</template>
<script lang='py'>
from vuepy import ref

# You can use the chat.value.send method to send messages
</script>


可以通过 send 方法发送消息：


In [ ]:
%%vuepy_run --plugins vpanel --show-code
<template>
<PnColumn>
  <PnChatFeed ref="chat" />
  <PnButton name="Send Message" @click="send_message" />
</PnColumn>
</template>
<script lang='py'>
from vuepy import ref

def send_message():
    chat.value.send(
        "Hello world!",
        user="Bot",
        avatar="B",
        footer_objects=[pn.widgets.Button(name="Footer Object")]
    )
</script>


## 消息回调

可以通过设置 callback 来创建更有趣的交互：


In [ ]:
%%vuepy_run --plugins vpanel --show-code
<template>
<PnChatFeed :callback="echo_message" />
</template>
<script lang='py'>
from vuepy import ref

def echo_message(contents, user, instance):
    return f"Echoing {user}... {contents}"
</script>


回调函数可以根据需要包含不同的参数：
- 只有一个参数时为 contents (消息内容)
- 两个参数时为 contents 和 user (用户名)
- 三个参数时为 contents、user 和 instance (组件实例)

可以通过设置 callback_user 和 callback_avatar 来修改响应者的默认名称和头像：


In [ ]:
%%vuepy_run --plugins vpanel --show-code
<template>
<PnChatFeed :callback="echo_message" callback-user="Echo Bot" callback-avatar="🛸" />
</template>
<script lang='py'>
from vuepy import ref

def echo_message(contents, user):
    return f"Echoing {user}... {contents}"
</script>


## 消息流式显示

通过 async generators 可以实现最简单和最理想的输出流式显示：


In [ ]:
%%vuepy_run --plugins vpanel --show-code
<template>
<PnChatFeed :callback="stream_message" />
</template>
<script lang='py'>
from vuepy import ref

async def stream_message(contents):
    message = ""
    for character in contents:
        message += character
        yield message
</script>


对于非生成器输出(比如LangChain输出)，也可以使用 stream 方法进行流式显示：


In [ ]:
%%vuepy_run --plugins vpanel --show-code
<template>
<PnColumn>
  <PnChatFeed ref="chat" />
  <PnButton name="Stream Message" @click="stream_demo" />
</PnColumn>
</template>
<script lang='py'>
from vuepy import ref
import time

def stream_demo():
    message = None
    for n in "123456789 abcdefghijklmnopqrstuvxyz":
        time.sleep(0.1)
        message = chat.value.stream(n, message=message)
</script>


## 自定义样式

可以通过 message_params 传递 ChatEntry 参数：


In [ ]:
%%vuepy_run --plugins vpanel --show-code
<template>
<PnChatFeed 
  :message-params="{
    'default_avatars': {'System': 'S', 'User': '👤'},
    'reaction_icons': {'like': 'thumb-up'}
  }"
  ref="chat"
/>
</template>
<script lang='py'>
from vuepy import ref

# Send some test messages
def mounted():
    chat.value.send(user="System", value="This is the System speaking.")
    chat.value.send(user="User", value="This is the User speaking.")
</script>


还可以通过 CSS 自定义消息外观：


In [ ]:
%%vuepy_run --plugins vpanel --show-code
<template>
<PnChatFeed 
  :show-activity-dot="true"
  :message-params="{
    'stylesheets': [
      '''
      .message {
          background-color: tan;
          font-family: \"Courier New\";
          font-size: 24px;
      }
      '''
    ]
  }"
  ref="chat"
/>
</template>
<script lang='py'>
from vuepy import ref

def mounted():
    chat.value.send("I am so stylish!")
</script>


## API

### 属性

| 属性名            | 说明                   | 类型                                                   | 默认值  |
| ---------------- | --------------------- | ----------------------------------------------------- | ------- |
| callback         | 消息回调函数           | ^[Callable]                                           | None    |
| callback_user    | 回调消息的默认用户名    | ^[str]                                                | —      |
| callback_avatar  | 回调消息的默认头像      | ^[str]                                               | —      |
| message_params   | ChatEntry 参数         | ^[dict]                                              | {}     |
| show_activity_dot| 显示活动状态点         | ^[bool]                                              | False  |
| height          | 组件高度              | ^[int \| str]                                         | —      |
| width           | 组件宽度              | ^[int \| str]                                         | —      |

### Events

| 事件名   | 说明                  | 类型                                     |
| ------- | -------------------- | ---------------------------------------- |
| message | 发送新消息时触发       | ^[Callable]`(message: dict) -> None`     |
| clear   | 清空消息时触发        | ^[Callable]`() -> None`                  |

### Slots

| 插槽名   | 说明               |
| ------- | ----------------- |
| default | 自定义默认内容      |

### 方法

| 方法名    | 说明                  | 参数                                    |
| -------- | ------------------- | --------------------------------------- |
| send     | 发送消息            | value, user, avatar, footer_objects     |
| stream   | 流式发送消息         | value, user, avatar, message            |
| clear    | 清空所有消息         | -                                       |
| undo     | 撤销最后的消息       | count: int = 1                          |


In [ ]:
##ignore
import panel as pn
pn.extension()

chat_feed = pn.chat.ChatFeed()
chat_feed.send(
    "Hello world!",
    user="Bot",
    avatar="B",
    footer_objects=[pn.widgets.Button(name="Footer Object")],
)